In [1]:
from package import *

import requests
from pprint import pprint
from functools import partial

pprint = partial(pprint, width = 200)


class CallApi:
	endpoints = {
		'videos':        'https://www.googleapis.com/youtube/v3/videos',
		'searchs':       'https://www.googleapis.com/youtube/v3/search',
		'channels':      'https://www.googleapis.com/youtube/v3/channels',
		'playlists':     'https://www.googleapis.com/youtube/v3/playlists',
		'playlistItems': 'https://www.googleapis.com/youtube/v3/playlistItems',
		'activities':    'https://www.googleapis.com/youtube/v3/activities',
		'watch':         'http://www.youtube.com/watch'
	}
	def __init__(self, **kwargs):
		pprint(dir(github))
		self.api_key = github.youtube_api_key


	def request(self, endpoint, **parameters):
		if not endpoint.endswith('s'): endpoint += 's'
		url = self.endpoints[endpoint]

		parameters['key'] = self.api_key

		response = requests.get(url, params = parameters)
		return response


test_api = CallApi()
youtube = YouTube(api_key = github.youtube_api_key)

playlist_parameters = {
	'id':         "PL1cXh4tWqmsEQPeLEJ5V3k5knt-X9k043",
	'maxResults': 50,
	'part':       "snippet,contentDetails",
}
search_parameters = {
	'playlistId': playlist_parameters['id'],
	'maxResults': 50,
	'part':       'snippet',
}
channel_parameters = {
	'id':   'UCkxctb0jr8vwa4Do6c6su0Q',
	'part': "snippet,statistics,topicDetails"
}
video_parameters = {
	'id':   'p9RKAalhtZM',
	'part': 'snippet,statistics,topicDetails'
}

response = test_api.request('channels', **channel_parameters)

#pprint(response.json(), width = 200)
print(type(response))
roosterteeth = 'UCzH3iADRIq1IJlIXjfNgTpA'
print("Finished!")


['DATA_FOLDER',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'github_folder',
 'os',
 'sys',
 'tabletools',
 'timetools',
 'youtube_api_key',
 'youtube_subscriptions']
<class 'requests.models.Response'>
Finished!


In [4]:
pprint(youtube.get('channel', roosterteeth).api_response.json())

{'etag': '"S8kisgyDEblalhHF9ooXPiFFrkc/X4O2fr6cqoZxyJGCVamtDg_z9U0"',
 'items': [{'contentDetails': {'relatedPlaylists': {'uploads': 'UUzH3iADRIq1IJlIXjfNgTpA', 'watchHistory': 'HL', 'watchLater': 'WL'}},
            'etag': '"S8kisgyDEblalhHF9ooXPiFFrkc/WkiM-NlSxJURoP0oB-CV68GlAHk"',
            'id': 'UCzH3iADRIq1IJlIXjfNgTpA',
            'kind': 'youtube#channel',
            'snippet': {'country': 'US',
                        'customUrl': 'roosterteeth',
                        'description': "Welcome to the Rooster Teeth Channel. We're a production company in Austin, TX, making podcasts, animated shows, and live-action shorts and series. We also "
                                       'make content on a bunch of gaming channels; check those out on the right. SUBSCRIBE: http://bit.ly/Roosterteeth',
                        'localized': {'description': "Welcome to the Rooster Teeth Channel. We're a production company in Austin, TX, making podcasts, animated shows, and live-action 

In [2]:
channel_items = youtube.getChannelItems(roosterteeth, verbose = True)
print(len(channel_items))

N/A% (0 of 7655) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

  1% (100 of 7655) |                      | Elapsed Time: 0:00:00 ETA:  0:00:31

  1% (150 of 7655) |                      | Elapsed Time: 0:00:00 ETA:  0:00:42

  2% (200 of 7655) |                      | Elapsed Time: 0:00:01 ETA:  0:00:48

  3% (250 of 7655) |                      | Elapsed Time: 0:00:01 ETA:  0:00:47

...

 98% (7550 of 7655) |#################### | Elapsed Time: 0:00:52 ETA:  0:00:00

 99% (7600 of 7655) |#################### | Elapsed Time: 0:00:52 ETA:  0:00:00

 99% (7650 of 7655) |#################### | Elapsed Time: 0:00:53 ETA:  0:00:00

100% (7655 of 7655) |#####################| Elapsed Time: 0:00:53 ETA:  0:00:00

7655


In [4]:
pprint(channel_items[0].toStandard())

{'channelId': 'UCzH3iADRIq1IJlIXjfNgTpA',
 'channelName': 'Rooster Teeth',
 'itemDescription': "Ruby, Weiss, Blake, and Yang are each entangled in journeys of their own, but they all share one destination: Haven Academy. Whether it's the promise of ancient relics, mystical "
                    "maidens, or simply more power, it's clear that the stage for the next great battle for Remnant has been chosen. The question is, with so many players in this game, who's going "
                    'to come out on top? Join FIRST to watch episodes early: http://bit.ly/2wf5zPJ\n'
                    '\n'
                    '» Get your Rooster Teeth merch: http://bit.ly/2uRi44x\n'
                    '» Subscribe: http://bit.ly/13y3Gum\n'
                    '\n'
                    'About RWBY Volume 5:\n'
                    "Ruby, Weiss, Blake, and Yang are each entangled in journeys of their own, but they all share one destination: Haven Academy. Whether it's the promise of ancient relics, m